In [1]:
from typing import Iterable
import numpy as np

from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs
from eval_process import evaluate

from evaluate.dataset import Dataset
from evaluate.eval_log import EvalLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.concurrent import TqdmPool

import mealpy

import config
import concurrent


OPTIMIZERS = [
    mealpy.swarm_based.PSO.OriginalPSO(c1=1, c2=2.05, w=0.2),
    mealpy.swarm_based.MSA.OriginalMSA(),
    mealpy.swarm_based.SCSO.OriginalSCSO(),
    mealpy.physics_based.SA.OriginalSA(),
    mealpy.physics_based.EVO.OriginalEVO(),
    mealpy.physics_based.EFO.DevEFO(),
    mealpy.physics_based.EO.ModifiedEO(),
    mealpy.human_based.ICA.OriginalICA(
        empire_count=7,
        assimilation_coeff=1.5,
        revolution_prob=0.4,
        revolution_rate=0.05,
        revolution_step_size=0.175,
    ),
    mealpy.human_based.FBIO.DevFBIO(),
    mealpy.human_based.SARO.OriginalSARO(se=0.5, mu=5),
    mealpy.evolutionary_based.GA.BaseGA(),
    mealpy.evolutionary_based.CRO.OCRO(),
    mealpy.evolutionary_based.DE.OriginalDE(strategy=0),
    mealpy.math_based.PSS.OriginalPSS(acceptance_rate=0.925),
    mealpy.math_based.SCA.DevSCA(),
    mealpy.math_based.HC.OriginalHC(neighbour_size=200),
]



In [2]:
exec = TqdmPool(max_workers=30)

run_config = MealRunConfig(max_time=15, silent=True, seed=0)

futures = []

for alg_name in ["UniformSampling", "IDUniformSampling", "RandomSampling"]:
    for obj_name in config.OBJECT_NAMES:

        future = exec.submit(
            evaluate,
            alg_name=alg_name,
            run_config=run_config,
            obj_name=obj_name,
            eval_data=config.EVAL_DATASET,
            log_folder=f"AllOptimizers2/{obj_name}",
        )

        futures.append(future)

# evaluate(alg_name='UniformSampling',
#         run_config=run_config,
#         obj_name='mug',
#         eval_data=config.EVAL_DATASET,
#         log_folder=f"runs/test"
#         )
for future in concurrent.futures.as_completed(futures):
    try:
        future.result()
    except Exception as e:
        print(e)

exec.shutdown(wait=True)


  0%|          | 0/1 [00:00<?, ?it/s]